In [ ]:
from glob import glob

ass_paths = glob("../data/Subtitles/*.ass")
srt_paths = glob("../data/Subtitles/*.srt")

subtitles_paths = sorted(ass_paths + srt_paths)

subtitles_paths[:5]

In [ ]:
import re

scripts = []
episode_num = []

for path in subtitles_paths:
    with open(path, "r", encoding="utf-8", errors="ignore") as file:
        lines = file.readlines()

        # Skip header lines if it's an .ass file
        if path.endswith(".ass"):
            lines = lines[27:]
            rows = [",".join(line.split(",")[9:]) for line in lines]
            rows = [line.replace("\\N", " ") for line in rows]
            script = " ".join(rows)

        elif path.endswith(".srt"):
            # Basic .srt parsing: skip subtitle numbers and timestamps
            lines = [line.strip() for line in lines if line.strip()]
            rows = [
                line
                for line in lines
                if not re.match(r"^\d+$", line)
                and not re.match(r"^\d{2}:\d{2}:\d{2},\d{3}", line)
            ]
            script = " ".join(rows)

        # Try to extract episode number using regex
        filename = path.split("/")[-1]
        match = re.search(r"\d+", filename)
        if match:
            episode = int(match.group())
            scripts.append(script)
            episode_num.append(episode)
        else:
            print(f"⚠️ Couldn't find episode number in filename: {filename}")

In [ ]:
%pip install pandas

In [ ]:
import pandas as pd

df = pd.DataFrame.from_dict({"episode": episode_num, "script": scripts})

In [ ]:
df.head()

In [ ]:
%pip install spacy

In [ ]:
%pip spacy download en_core_web_trf


In [ ]:
import spacy
nlp = spacy.load("en_core_web_trf")